<a href="https://colab.research.google.com/github/guru3/soduku_solver/blob/master/2.%20Better%20Training%20Approach%20%3A%20Can%20a%20neural%20network%20solve%20the%20soduku%20game%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Let's change the approach to train our network

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"guru333","key":"695a60acdd9541c267fbd36712d5ffcd"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download bryanpark/sudoku
!unzip sudoku.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
sudoku.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  sudoku.zip
  inflating: sudoku.csv              


In [3]:
import csv
import random
import pickle
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Conv2D,Dense,Flatten,Input,Dropout
import matplotlib.pyplot as plt
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
def read_raw_data(total_to_read):
    FILE_PATH = './sudoku.csv';
    soduku_games = csv.reader(open(FILE_PATH,'r'))
    next(soduku_games);

    quizzes = [];
    solutions = [];
    index = 0;
    for game in soduku_games:
        index = index+ 1;
        if( index == total_to_read ):
            break;
        
        quizzes.append( np.reshape([int(d) for d in game[0]], (9, 9)) )
        solutions.append( np.reshape([int(d) for d in game[1]], (9, 9)) )
        
    permutation = np.random.permutation(len(quizzes));
    quizzes = np.array(quizzes)[permutation];
    solutions = np.array(solutions)[permutation];
    return quizzes, solutions;

### We will try to manually increase the unsolved cells while maintaining the given clue count above or equal to 17 ( https://www.technologyreview.com/2012/01/06/188520/mathematicians-solve-minimum-sudoku-problem/ )

In [0]:
def change_output(Y):
    return [ Y[:,i,j,:] for i in range(9) for j in range(9) ]

def remove_clues(inputs, clues_to_remove=0):
    for input in inputs:
      clue_removed = 0;
      can_remove = [];
      for row in range(9):
        for col in range(9):
          if( np.sum(input[row,col,:]) != 0 ):
            can_remove.append( [row,col] )
      
      can_remove = np.array(can_remove)[ np.random.permutation(len(can_remove)) ];
      while(clue_removed < clues_to_remove):
        [row,col] = can_remove[clue_removed];
        input[row,col,:] = 0;
        clue_removed = clue_removed + 1;
    return inputs

def data_to_categorical(): # we will use one hot encoding of input and output  - this breakdown is more intuitive in the given case of discrete numbers
    X, Y = read_raw_data(50000);
    
    X = to_categorical(X, num_classes=10)
    X_also = to_categorical(Y, num_classes=10);
    Y = to_categorical(Y-1, num_classes=9)
    total_ex = X.shape[0];
    training_ex = (int)(total_ex*0.8);
    validation_ex= (int)(total_ex*0.1);
    train_X, train_Y = X_also[:training_ex], change_output(Y[:training_ex]);
    valid_X, valid_Y = X[training_ex:training_ex+validation_ex], change_output(Y[training_ex:training_ex+validation_ex]);
    test_X, test_Y = X[training_ex+validation_ex:], change_output(Y[training_ex+validation_ex:]);
    
    return X_also[:training_ex], train_Y, valid_X, valid_Y, test_X, test_Y;

In [0]:
train_X,train_Y,valid_X,valid_Y,test_X,test_Y = data_to_categorical();

In [7]:
print(train_X.shape)
print(len(train_Y),train_Y[0].shape)

(39999, 9, 9, 10)
81 (39999, 9)


In [0]:
def get_model():
    model = Sequential();
    model.add( Dense(128, input_shape=(9,9,10)) );
    model.add( Dropout(0.5) );
    model.add( Dense(64) );
    model.add( Flatten() )
    
    input_ = Input(shape=(9,9,10))
    intermediate_output = model(input_)
    
    outputs = [ Dense(9, activation='sigmoid')(intermediate_output) for i in range(81) ] #for each cell in sudoku table
    model = Model(input_, outputs);
    model.compile( optimizer='AdaDelta', loss='categorical_crossentropy', metrics= ['accuracy'] )
    return model

In [0]:
model = get_model();

In [13]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 9, 9, 10)     0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 5184)         9664        input_2[0][0]                    
__________________________________________________________________________________________________
dense_86 (Dense)                (None, 9)            46665       sequential_2[1][0]               
__________________________________________________________________________________________________
dense_87 (Dense)                (None, 9)            46665       sequential_2[1][0]               
____________________________________________________________________________________________

In [15]:
train_X_round = remove_clues(train_X, 0);
history = model.fit(train_X_round, train_Y, epochs=3, validation_data=(valid_X, valid_Y))

Train on 39999 samples, validate on 4999 samples
Epoch 1/3
39999/39999 [==============================] - 219s 5ms/step - loss: 4.9858 - dense_86_loss: 0.0630 - dense_87_loss: 0.0624 - dense_88_loss: 0.0621 - dense_89_loss: 0.0609 - dense_90_loss: 0.0614 - dense_91_loss: 0.0622 - dense_92_loss: 0.0595 - dense_93_loss: 0.0610 - dense_94_loss: 0.0627 - dense_95_loss: 0.0621 - dense_96_loss: 0.0603 - dense_97_loss: 0.0620 - dense_98_loss: 0.0626 - dense_99_loss: 0.0609 - dense_100_loss: 0.0628 - dense_101_loss: 0.0611 - dense_102_loss: 0.0602 - dense_103_loss: 0.0619 - dense_104_loss: 0.0616 - dense_105_loss: 0.0596 - dense_106_loss: 0.0626 - dense_107_loss: 0.0616 - dense_108_loss: 0.0628 - dense_109_loss: 0.0627 - dense_110_loss: 0.0617 - dense_111_loss: 0.0608 - dense_112_loss: 0.0622 - dense_113_loss: 0.0610 - dense_114_loss: 0.0624 - dense_115_loss: 0.0627 - dense_116_loss: 0.0615 - dense_117_loss: 0.0598 - dense_118_loss: 0.0626 - dense_119_loss: 0.0606 - dense_120_loss: 0.0610 - de

In [18]:
train_X_round = remove_clues(train_X, 5);
history = model.fit(train_X_round, train_Y, epochs=10, validation_data=(valid_X, valid_Y))

Train on 39999 samples, validate on 4999 samples
Epoch 1/10
39999/39999 [==============================] - 204s 5ms/step - loss: 2.3911 - dense_86_loss: 0.0286 - dense_87_loss: 0.0331 - dense_88_loss: 0.0301 - dense_89_loss: 0.0287 - dense_90_loss: 0.0312 - dense_91_loss: 0.0311 - dense_92_loss: 0.0302 - dense_93_loss: 0.0284 - dense_94_loss: 0.0288 - dense_95_loss: 0.0294 - dense_96_loss: 0.0318 - dense_97_loss: 0.0300 - dense_98_loss: 0.0305 - dense_99_loss: 0.0284 - dense_100_loss: 0.0303 - dense_101_loss: 0.0285 - dense_102_loss: 0.0283 - dense_103_loss: 0.0279 - dense_104_loss: 0.0296 - dense_105_loss: 0.0300 - dense_106_loss: 0.0276 - dense_107_loss: 0.0284 - dense_108_loss: 0.0307 - dense_109_loss: 0.0303 - dense_110_loss: 0.0285 - dense_111_loss: 0.0304 - dense_112_loss: 0.0294 - dense_113_loss: 0.0293 - dense_114_loss: 0.0280 - dense_115_loss: 0.0262 - dense_116_loss: 0.0300 - dense_117_loss: 0.0303 - dense_118_loss: 0.0290 - dense_119_loss: 0.0293 - dense_120_loss: 0.0322 - d

In [19]:
train_X_round = remove_clues(train_X, 10);
history = model.fit(train_X_round, train_Y, epochs=10, validation_data=(valid_X, valid_Y))

Train on 39999 samples, validate on 4999 samples
Epoch 1/10
39999/39999 [==============================] - 206s 5ms/step - loss: 5.6799 - dense_86_loss: 0.0735 - dense_87_loss: 0.0736 - dense_88_loss: 0.0665 - dense_89_loss: 0.0698 - dense_90_loss: 0.0721 - dense_91_loss: 0.0738 - dense_92_loss: 0.0708 - dense_93_loss: 0.0697 - dense_94_loss: 0.0685 - dense_95_loss: 0.0733 - dense_96_loss: 0.0662 - dense_97_loss: 0.0684 - dense_98_loss: 0.0670 - dense_99_loss: 0.0699 - dense_100_loss: 0.0732 - dense_101_loss: 0.0735 - dense_102_loss: 0.0697 - dense_103_loss: 0.0718 - dense_104_loss: 0.0697 - dense_105_loss: 0.0681 - dense_106_loss: 0.0688 - dense_107_loss: 0.0712 - dense_108_loss: 0.0713 - dense_109_loss: 0.0713 - dense_110_loss: 0.0729 - dense_111_loss: 0.0710 - dense_112_loss: 0.0711 - dense_113_loss: 0.0715 - dense_114_loss: 0.0688 - dense_115_loss: 0.0723 - dense_116_loss: 0.0677 - dense_117_loss: 0.0688 - dense_118_loss: 0.0703 - dense_119_loss: 0.0660 - dense_120_loss: 0.0708 - d

In [20]:
train_X_round = remove_clues(train_X, 20);
history = model.fit(train_X_round, train_Y, epochs=20, validation_data=(valid_X, valid_Y))

Train on 39999 samples, validate on 4999 samples
Epoch 1/20
39999/39999 [==============================] - 205s 5ms/step - loss: 28.1616 - dense_86_loss: 0.3493 - dense_87_loss: 0.3538 - dense_88_loss: 0.3541 - dense_89_loss: 0.3474 - dense_90_loss: 0.3463 - dense_91_loss: 0.3481 - dense_92_loss: 0.3479 - dense_93_loss: 0.3471 - dense_94_loss: 0.3451 - dense_95_loss: 0.3520 - dense_96_loss: 0.3488 - dense_97_loss: 0.3497 - dense_98_loss: 0.3454 - dense_99_loss: 0.3429 - dense_100_loss: 0.3469 - dense_101_loss: 0.3489 - dense_102_loss: 0.3509 - dense_103_loss: 0.3483 - dense_104_loss: 0.3488 - dense_105_loss: 0.3477 - dense_106_loss: 0.3541 - dense_107_loss: 0.3548 - dense_108_loss: 0.3453 - dense_109_loss: 0.3458 - dense_110_loss: 0.3517 - dense_111_loss: 0.3509 - dense_112_loss: 0.3421 - dense_113_loss: 0.3437 - dense_114_loss: 0.3457 - dense_115_loss: 0.3451 - dense_116_loss: 0.3557 - dense_117_loss: 0.3498 - dense_118_loss: 0.3471 - dense_119_loss: 0.3443 - dense_120_loss: 0.3415 - 

In [21]:
history = model.fit(train_X_round, train_Y, epochs=20, validation_data=(valid_X, valid_Y))

Train on 39999 samples, validate on 4999 samples
Epoch 1/20
39999/39999 [==============================] - 204s 5ms/step - loss: 18.7965 - dense_86_loss: 0.2344 - dense_87_loss: 0.2377 - dense_88_loss: 0.2380 - dense_89_loss: 0.2347 - dense_90_loss: 0.2312 - dense_91_loss: 0.2313 - dense_92_loss: 0.2321 - dense_93_loss: 0.2350 - dense_94_loss: 0.2319 - dense_95_loss: 0.2372 - dense_96_loss: 0.2350 - dense_97_loss: 0.2344 - dense_98_loss: 0.2280 - dense_99_loss: 0.2266 - dense_100_loss: 0.2294 - dense_101_loss: 0.2321 - dense_102_loss: 0.2335 - dense_103_loss: 0.2352 - dense_104_loss: 0.2337 - dense_105_loss: 0.2282 - dense_106_loss: 0.2354 - dense_107_loss: 0.2358 - dense_108_loss: 0.2296 - dense_109_loss: 0.2308 - dense_110_loss: 0.2371 - dense_111_loss: 0.2351 - dense_112_loss: 0.2290 - dense_113_loss: 0.2256 - dense_114_loss: 0.2309 - dense_115_loss: 0.2338 - dense_116_loss: 0.2353 - dense_117_loss: 0.2328 - dense_118_loss: 0.2314 - dense_119_loss: 0.2311 - dense_120_loss: 0.2260 - 

KeyboardInterrupt: ignored

In [0]:
output = model.predict(test_X[0:1])

In [28]:
unsolved = np.array([ np.argmax(test_X[0,i,j,:]) for i in range(9) for j in range(9) ] ).reshape(9,9);
unsolved

array([[0, 0, 3, 0, 0, 0, 0, 0, 0],
       [0, 8, 0, 0, 0, 0, 9, 1, 5],
       [6, 0, 1, 2, 8, 0, 7, 0, 0],
       [0, 0, 4, 0, 0, 7, 1, 6, 3],
       [0, 0, 6, 0, 9, 2, 0, 0, 0],
       [7, 0, 0, 1, 0, 0, 4, 0, 0],
       [8, 0, 0, 5, 0, 4, 0, 7, 1],
       [2, 3, 0, 0, 0, 9, 0, 0, 8],
       [0, 4, 0, 6, 3, 0, 0, 2, 0]])

In [38]:
solved = np.array([ (np.argmax(output[i]) + 1) for i in range(81) ]).reshape(9,9);
solved

array([[5, 2, 3, 9, 5, 5, 2, 8, 2],
       [4, 8, 2, 7, 6, 3, 9, 1, 5],
       [6, 9, 1, 2, 8, 5, 7, 3, 4],
       [5, 2, 4, 8, 5, 7, 1, 6, 3],
       [1, 5, 6, 4, 9, 2, 8, 5, 7],
       [7, 2, 8, 1, 6, 6, 4, 5, 2],
       [8, 6, 9, 5, 2, 4, 3, 7, 1],
       [2, 3, 7, 7, 1, 9, 5, 5, 8],
       [5, 4, 5, 6, 3, 8, 5, 2, 9]])

In [39]:
actual_solution = np.array([ (np.argmax(np.array(test_Y)[i,0,:]) + 1 ) for i in range(81) ] ).reshape(9,9);
actual_solution

array([[5, 7, 3, 9, 4, 1, 2, 8, 6],
       [4, 8, 2, 3, 7, 6, 9, 1, 5],
       [6, 9, 1, 2, 8, 5, 7, 3, 4],
       [9, 2, 4, 8, 5, 7, 1, 6, 3],
       [3, 1, 6, 4, 9, 2, 8, 5, 7],
       [7, 5, 8, 1, 6, 3, 4, 9, 2],
       [8, 6, 9, 5, 2, 4, 3, 7, 1],
       [2, 3, 5, 7, 1, 9, 6, 4, 8],
       [1, 4, 7, 6, 3, 8, 5, 2, 9]])

### We can clearly see solution being incorrect for given test input. Given lack of fast processing units, I will not continuing this experiment. These are times when I want to go out and buy a super fast GPU!